## E1 Proyecto Minería de datos
### Luis Alan Morales Castillo A01659147
### Paulina Díaz Arroyo A01029592
### Mario Gaitan Reyna A01659057


## Importar librerias

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV,train_test_split
import plotly.express as px

## Lectura de datos

In [3]:
# Leer el archivo CSV con los datos y eliminar filas vacías
df= pd.read_csv('../datos/Amazon_Unlocked_Mobile.csv')
df.dropna(inplace=True)
# Las reseñas con puntuación >= 4 se marcan como 1 (positiva)
# Las demás como 0 (negativa)
df["Positividad"] = df["Rating"].map(lambda x: 1 if x >= 4 else 0)

## SVC

In [24]:
# Dividir los datos en entrenamiento y prueba 
x= df['Reviews']
y= df['Positividad']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=39,stratify=y)

In [ ]:
# Crear el pipeline de procesamiento y entrenamiento
pipeline= Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('svm', LinearSVC(max_iter=1000)) 
])

In [ ]:
# Ajustar el modelo con los datos de entrenamiento
pipeline.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=10000)),
                ('svm', LinearSVC(random_state=39))])

In [ ]:
# Realizar predicciones y mostrar el reporte de clasificación
pred=pipeline.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.87     41464
           1       0.93      0.95      0.94     92268

    accuracy                           0.92    133732
   macro avg       0.91      0.90      0.90    133732
weighted avg       0.92      0.92      0.92    133732



El output generado en la celda anterior muestra un DataFrame que contiene los resultados del proceso de búsqueda de hiperparámetros realizado con GridSearchCV. Este DataFrame incluye información detallada sobre cada combinación de parámetros evaluada, junto con las métricas de rendimiento obtenidas. Se puede observar que la categoria 0 tiene una precision de 0.88, mientras que su contraparte obtiene una precision de 0.93

## SVC Ajustes Manuales

In [ ]:
# Crear el pipeline de procesamiento y entrenamiento
pipeline2= Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000)),
    ('svm', LinearSVC(max_iter=2000)) 
])

In [43]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x, y, test_size=0.6,stratify=y)
pipeline2.fit(x_train2, y_train2)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=20000)),
                ('svm', LinearSVC(max_iter=2000))])

In [44]:
# Realizar predicciones y mostrar el reporte de clasificación
pred=pipeline2.predict(x_test2)
print(classification_report(y_test2, pred))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87     62195
           1       0.93      0.95      0.94    138402

    accuracy                           0.92    200597
   macro avg       0.91      0.90      0.91    200597
weighted avg       0.92      0.92      0.92    200597



Modificando el parametro de TfidfVectorizer max_features y subiendo las máximas iteraciones de LinearSVC, se logra que la clase 0 aumente en accuracy un 0.1, mejorando el modelo, cabe a resaltar que tambien se cambio la proporcion de partición de los datos de 0.4 a 0.6

## GridSearchCV

In [ ]:
#definir paramtros para la búsqueda de hiperparámetros
parameters = {'svm__loss':('hinge', 'squared_hinge'), 'svm__C':[1, 10],
              'svm__multi_class': ('ovr', 'crammer_singer'),
              'tfidf__max_features':[1000,10000],
              'svm__random_state':[23,15]}
clf= GridSearchCV(pipeline, parameters, scoring='accuracy', verbose=1)
# Ajustar el modelo con los datos de entrenamiento
clf.fit(x_train, y_train)

In [ ]:
# Mostrar los mejores parámetros encontrados
df_res=pd.DataFrame(clf.cv_results_)
df_res.sort_values(by="rank_test_score",ascending=True).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svm__C,param_svm__loss,param_svm__multi_class,param_svm__random_state,param_tfidf__max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,21.898143,7.327802,0.653043,0.058769,1,hinge,crammer_singer,15,10000,"{'svm__C': 1, 'svm__loss': 'hinge', 'svm__mult...",0.916077,0.916997,0.916598,0.917770,0.916972,0.916883,0.000555,1
5,34.932051,16.301588,0.672473,0.038015,1,hinge,crammer_singer,23,10000,"{'svm__C': 1, 'svm__loss': 'hinge', 'svm__mult...",0.916077,0.916997,0.916598,0.917770,0.916972,0.916883,0.000555,1
13,28.697941,16.259204,0.690886,0.027610,1,squared_hinge,crammer_singer,23,10000,"{'svm__C': 1, 'svm__loss': 'squared_hinge', 's...",0.916077,0.916997,0.916598,0.917770,0.916972,0.916883,0.000555,1
15,20.665200,7.682458,0.652094,0.054355,1,squared_hinge,crammer_singer,15,10000,"{'svm__C': 1, 'svm__loss': 'squared_hinge', 's...",0.916077,0.916997,0.916598,0.917770,0.916972,0.916883,0.000555,1
19,7.825647,0.388946,0.669261,0.023716,10,hinge,ovr,15,10000,"{'svm__C': 10, 'svm__loss': 'hinge', 'svm__mul...",0.916949,0.915800,0.916897,0.917122,0.916548,0.916663,0.000470,5


In [ ]:
#df_res.to_csv('datos/resultados.csv', index=False)
sorted(clf.best_params_.items())

[('svm__C', 1),
 ('svm__loss', 'hinge'),
 ('svm__multi_class', 'crammer_singer'),
 ('svm__random_state', 23),
 ('tfidf__max_features', 10000)]


En la implementación de GridSearchCV se realizó una búsqueda exhaustiva de hiperparámetros para optimizar el rendimiento del modelo SVC. Se evaluaron diferentes combinaciones de parámetros, incluyendo el tipo de pérdida (hinge y squared_hinge), el valor de regularización C (1 y 10), el método de clasificación (ovr y crammer_singer), el número máximo de características del vectorizador Tfidf (1000 y 10000), y el estado aleatorio del modelo. Los resultados muestran que la mejor combinación de parámetros logró una precisión promedio de 0.916883, utilizando `hinge` como función de pérdida, `C=1`, el método `crammer_singer`, y un máximo de 10000 características en el vectorizador. Este ajuste permitió mejorar la capacidad predictiva del modelo, destacando la importancia de seleccionar adecuadamente los hiperparámetros para obtener un rendimiento óptimo. Además, se observó que el tiempo de ajuste varía significativamente dependiendo de la configuración de los parámetros. 


### Gräficas

La gráfica presentada es un diagrama de dispersión que permite analizar la relación entre el tiempo promedio de ajuste (mean_fit_time) y la puntuación promedio de prueba (mean_test_score). Los puntos están coloreados según el número máximo de características del vectorizador Tfidf (param_tfidf__max_features). Se observa que existe una separacion marcada y que con 10k features se eleva el score y tambien el fit time en casos marcados

In [62]:
fig = px.scatter(
    df_res,
    x="mean_test_score",
    y="mean_fit_time",
    color="param_tfidf__max_features",
    title="Mean Test Score vs Mean fit time",
    labels={"rank_test_score": "Rank Test Score", "mean_test_score": "Mean Test Score", "param_tfidf__max_features": "Max Features"},
    hover_data=["params"]
)
fig.show()


La gráfica presentada es un diagrama de coordenadas paralelas que permite visualizar la relación entre los hiperparámetros del modelo y los resultados obtenidos durante el proceso de ajuste. Cada línea representa una combinación de parámetros evaluada, mientras que las dimensiones corresponden a métricas como el tiempo promedio de ajuste (mean_fit_time), la puntuación promedio de prueba (mean_test_score), el valor de regularización (C) y el número máximo de características (max_features). El color de las líneas indica el rango de la puntuación de prueba (rank_test_score), facilitando la identificación de las configuraciones más óptimas.


In [76]:
fig = px.parallel_coordinates(
    df_res,
    dimensions=["mean_fit_time", "mean_test_score", "param_svm__C", "param_tfidf__max_features"],
    color="rank_test_score",
    labels={
        "mean_fit_time": "mean_fit_time",
        "mean_test_score": "mean_test_score",
        "param_svm__C": "C",
        "param_tfidf__max_features": "max_features",
        "rank_test_score": "rank_test_score"
    },
    title="Coordenadas Paralelas de los hiperparámetros y resultados del modelo",
)
fig.show()

La gráfica de caja presentada muestra la variabilidad del tiempo de puntuación (std_score_time) en función del número máximo de características del vectorizador Tfidf (param_tfidf__max_features) y el valor de regularización (param_svm__C). Cada caja representa la distribución de los valores de (std_score_time) para una combinación específica de estos parámetros. Las cajas están coloreadas según el valor de param_svm__C Además, las cajas permiten identificar visualmente diferencias significativas entre medianas como se muestra significativamente dependiendo de la cantidad de features


In [75]:
fig = px.box(
    df_res,
    x="param_tfidf__max_features",
    y="std_score_time",
    color="param_svm__C",
    title="Variabilidad del puntaje de ajuste por Max Features y C",
    labels={
        "param_tfidf__max_features": "max_features",
        "std_score_time": "std_score_time",
        "param_svm__C": "C"
    },notched=True
)
fig.show()